In [1]:
import torch, data
import torch.nn as nn
import torch.optim as optim
import sparseconvnet as scn
import time
import os, sys
import math
import numpy as np

In [2]:
data.init(24,24*8,1)
dimension = 3
reps = 1 #Conv block repetition factor
m = 32 #Unet number of features
nPlanes = [m, 2*m, 3*m, 4*m, 5*m] #UNet number of features per level

4 training samples


In [3]:
print(data.nClassesTotal)

25


In [4]:
class Model(nn.Module):
    def __init__(self):
        nn.Module.__init__(self)
        self.sparseModel = scn.Sequential().add(
           scn.InputLayer(dimension, data.spatialSize, mode=3)).add(
           scn.SubmanifoldConvolution(dimension, 1, m, 3, False)).add(
           scn.UNet(dimension, reps, nPlanes, residual_blocks=False, downsample=[2,2])).add(
           scn.BatchNormReLU(m)).add(
           scn.OutputLayer(dimension))
        self.linear = nn.Linear(m, data.nClassesTotal)
    def forward(self,x):
        x=self.sparseModel(x)
        x=self.linear(x)
        return x

model=Model()
print(model)

Model(
  (sparseModel): Sequential(
    (0): InputLayer()
    (1): SubmanifoldConvolution 1->32 C3
    (2): Sequential(
      (0): Sequential(
        (0): BatchNormLeakyReLU(32,eps=0.0001,momentum=0.9,affine=True,leakiness=0)
        (1): SubmanifoldConvolution 32->32 C3
      )
      (1): ConcatTable(
        (0): Identity()
        (1): Sequential(
          (0): BatchNormLeakyReLU(32,eps=0.0001,momentum=0.9,affine=True,leakiness=0)
          (1): Convolution 32->64 C2/2
          (2): Sequential(
            (0): Sequential(
              (0): BatchNormLeakyReLU(64,eps=0.0001,momentum=0.9,affine=True,leakiness=0)
              (1): SubmanifoldConvolution 64->64 C3
            )
            (1): ConcatTable(
              (0): Identity()
              (1): Sequential(
                (0): BatchNormLeakyReLU(64,eps=0.0001,momentum=0.9,affine=True,leakiness=0)
                (1): Convolution 64->96 C2/2
                (2): Sequential(
                  (0): Sequential(
             

In [5]:
trainIterator=data.train()
validIterator=data.valid()

['../../dataset/machining_feature/points/0-0-23.pts', '../../dataset/machining_feature/points/0-4-23.pts', '../../dataset/machining_feature/points/0-8-23.pts', '../../dataset/machining_feature/points/0-7-23.pts']
['../../dataset/machining_feature/points/0-6-23.pts', '../../dataset/machining_feature/points/0-2-23.pts', '../../dataset/machining_feature/points/0-1-23.pts', '../../dataset/machining_feature/points/0-3-23.pts']


In [6]:
batch = next(iter(trainIterator))
for x in batch['x']:
    print(x[200:210])
print(batch['y'][200:210])
for xf in batch['xf']:
    print(xf)
for n in batch['nPoints']:
    print(n)


tensor([[102,  70, 131,   0],
        [102,  70, 131,   0],
        [102,  71, 131,   0],
        [101,  71, 131,   0],
        [101,  71, 131,   0],
        [101,  71, 131,   0],
        [101,  71, 131,   0],
        [100,  71, 131,   0],
        [100,  71, 131,   0],
        [103,  70, 131,   0]])
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]])
tensor([24, 24, 24, 24, 24, 24, 24, 24, 24, 24])
../../dataset/machining_feature/points/0-8-23.pts
26480


In [10]:
criterion = nn.CrossEntropyLoss()
p={}
p['n_epochs'] = 1
p['initial_lr'] = 1e-1
p['lr_decay'] = 4e-2
p['weight_decay'] = 1e-4
p['momentum'] = 0.9
p['check_point'] = False
p['use_cuda'] = torch.cuda.is_available()
dtype = 'torch.cuda.FloatTensor' if p['use_cuda'] else 'torch.FloatTensor'
dtypei = 'torch.cuda.LongTensor' if p['use_cuda'] else 'torch.LongTensor'
if p['use_cuda']:
    model.cuda()
    criterion.cuda()
optimizer = optim.SGD(model.parameters(),
    lr=p['initial_lr'],
    momentum = p['momentum'],
    weight_decay = p['weight_decay'],
    nesterov=True)
if p['check_point'] and os.path.isfile('epoch.pth'):
    p['epoch'] = torch.load('epoch.pth') + 1
    print('Restarting at epoch ' +
          str(p['epoch']) +
          ' from model.pth ..')
    model.load_state_dict(torch.load('model.pth'))
else:
    p['epoch']=1
print(p)
print('#parameters', sum([x.nelement() for x in model.parameters()]))

{'n_epochs': 1, 'initial_lr': 0.1, 'lr_decay': 0.04, 'weight_decay': 0.0001, 'momentum': 0.9, 'check_point': False, 'use_cuda': False, 'epoch': 1}
#parameters 3840409


In [8]:
def store(stats,batch,predictions,loss):
    ctr=0
    for nP,f in zip(batch['nPoints'],batch['xf']):
        f=f.split('/')[-1]
        if not f in stats:
            stats[f]={'p': 0, 'y': 0}
        #print(predictions[ctr:ctr+nP,classOffset:classOffset+nClasses].abs().max().item())
        stats[f]['p']+=predictions.detach()[ctr:ctr+nP].cpu().numpy()
        stats[f]['y']=batch['y'].detach()[ctr:ctr+nP].cpu().numpy()
        ctr+=nP

def inter(pred, gt, label):
    assert pred.size == gt.size, 'Predictions incomplete!'
    return np.sum(np.logical_and(pred.astype('int') == label, gt.astype('int') == label))

def union(pred, gt, label):
    assert pred.size == gt.size, 'Predictions incomplete!'
    return np.sum(np.logical_or(pred.astype('int') == label, gt.astype('int') == label))

def iou(stats):
    eps = sys.float_info.epsilon
    
    nmodels = len(stats)
    pred = []
    gt = []
    for j in stats.values():
        pred.append(j['p'].argmax(1))
        gt.append(j['y'])
    npart = np.max(np.concatenate(gt))+1
    iou_per_part = np.zeros((len(pred), npart))
    # loop over parts
    for j in range(npart):
        # loop over CAD models
        for k in range(len(pred)):
            p = pred[k]
            iou_per_part[k, j] = (inter(p, gt[k], j) + eps) / (union(p, gt[k], j) + eps)
    # average over CAD models and parts
    iou_all = np.mean(iou_per_part)
    return {'nmodels_sum': nmodels, 'iou': iou_all}

In [11]:
for epoch in range(p['epoch'], p['n_epochs'] + 1):
    model.train()
    stats = {}
    for param_group in optimizer.param_groups:
        param_group['lr'] = p['initial_lr'] * \
        math.exp((1 - epoch) * p['lr_decay'])
    scn.forward_pass_multiplyAdd_count=0
    scn.forward_pass_hidden_states=0
    start = time.time()
    for batch in trainIterator:
        optimizer.zero_grad()
        batch['x'][1]=batch['x'][1].type(dtype)
        batch['y']=batch['y'].type(dtypei)
        predictions=model(batch['x'])
        print(len(predictions))
        loss = criterion.forward(predictions,batch['y'])
        store(stats,batch,predictions,loss)
        loss.backward()
        optimizer.step()
    r = iou(stats)
    print('train epoch',epoch,1,'iou=', r['iou'], 'MegaMulAdd=',scn.forward_pass_multiplyAdd_count/r['nmodels_sum']/1e6, 'MegaHidden',scn.forward_pass_hidden_states/r['nmodels_sum']/1e6,'time=',time.time() - start,'s')

    if p['check_point']:
        torch.save(epoch, 'epoch.pth')
        torch.save(model.state_dict(),'model.pth')

    if epoch in [10,30,100]:
        model.eval()
        stats = {}
        scn.forward_pass_multiplyAdd_count=0
        scn.forward_pass_hidden_states=0
        start = time.time()
        for rep in range(1,1+3):
            for batch in validIterator:
                batch['x'][1]=batch['x'][1].type(dtype)
                batch['y']=batch['y'].type(dtypei)
                predictions=model(batch['x'])
                loss = criterion.forward(predictions,batch['y'])
                store(stats,batch,predictions,loss)
            r = iou(stats)
            print('valid epoch',epoch,rep,'iou=', r['iou'], 'MegaMulAdd=',scn.forward_pass_multiplyAdd_count/r['nmodels_sum']/1e6, 'MegaHidden',scn.forward_pass_hidden_states/r['nmodels_sum']/1e6,'time=',time.time() - start,'s')
        print(r['iou'])


37702
26480
32148
31632
train epoch 1 1 iou= 0.9048107445534499 MegaMulAdd= 223.485456 MegaHidden 0.369216 time= 0.6727063655853271 s
